In [486]:
import os
import math
import numpy as np
from matplotlib import pyplot
%matplotlib inline

In [487]:
#vocabulary
file1 = open("Dataset/positive-words.txt", "r",encoding='ISO-8859-1')
posText = file1.readlines()

file2 = open("Dataset/negative-words.txt", "r",encoding='ISO-8859-1')
negText = file2.readlines()

pos_vocab = list()
neg_vocab = list()

testdataset = []
traindataset = []


for line in posText:
    pos_vocab.append(line.strip())
       
#print(pos_vocab) 
for line in negText:
    neg_vocab.append(line.strip())
#print(neg_vocab)


In [488]:
def findStrings(s1,s2):
    s3 = set(s1)&set(s2)
    return s3


def findNO(s1):
    for index in range(len(s1)):
        if s1[index] == "no":
            return 1
        
    return 0  


def findEXC(s1):
    for index in range(len(s1)):
        if s1[index] == "!":
            return 1
        
    return 0  

In [489]:

def preprocessing(datapath,Y):
    
    numberofpositives = []
    numberofnegatives =[]
    rating = []
    processeddata = []
    isNO = []
    isEXC =[]
    count = []
    output = []
    
    processeddata = []
    for file in os.listdir(datapath):
            #print(file)
            fileopened = open(datapath+ "/" + file, "r",encoding='ISO-8859-1')
            text = fileopened.read()
            strings = text.split(' ')
            #print(strings) 
            #print(rating)
            #print(numberofwords)
            #print(numberofpositives)
            numberofpositives.append((len(findStrings(strings,pos_vocab))))
            numberofnegatives.append(len(findStrings(strings,neg_vocab)))
            ratingsplit = (file.split("_")[1])
            rating.append(ratingsplit.split(".")[0])
            isNO.append(findNO(strings))
            isEXC.append(findEXC(strings))
            count.append(len(strings))
            output.append(Y)
            
            dataset = np.vstack([numberofpositives,numberofnegatives,rating,count,isNO,isEXC,output]).T
            #print('dataset is',dataset)

    #print('dataset is',dataset)
    fileopened.close()
    return dataset


In [490]:
#print('Positives')
traindatasetpos =[]
traindatasetneg = []
traindatasetpos = preprocessing("Dataset/train/pos","1")
#print(traindatasetpos)
#print('negatives')
traindatasetneg = preprocessing("Dataset/train/neg","0")
#print(traindatasetneg)
traindatasetall = np.concatenate((traindatasetpos, traindatasetneg))
print(traindatasetall)

[['2' '0' '9' ... '1' '0' '1']
 ['10' '8' '8' ... '0' '0' '1']
 ['3' '0' '7' ... '0' '0' '1']
 ...
 ['2' '3' '1' ... '0' '0' '0']
 ['12' '15' '3' ... '0' '0' '0']
 ['5' '2' '3' ... '0' '0' '0']]


In [491]:
X, Y = traindatasetall[:, 0:6], traindatasetall[:, 6]
X = X.astype(np.float)
Y = Y.astype(np.float)

#normalization
means_X = np.mean(X,axis=0)
sd_X = np.std(X,axis = 0)
for i in range (0,len(X)):
    
    for j in range(0,6):
        X[i][j]= (X[i][j]-means_X[j])/sd_X[j] 

X0 = np.ones((len(X),1))
X = np.hstack((X0,X))

#print(X)
#print(Y)

In [492]:
#sigmoid func
def sigmoid(X):
    sig = 1/(1+ np.exp(-X))
    return sig

In [493]:
#loss functn
def lossfunt(X,Y,thetas): 
    m = Y.size 
    J = 0
    S1 = 0
    S2 = 0
    
    
    for i in range (0,m):  
        dot = np.dot(X[i],thetas)
        Sigmoid = sigmoid(dot)
        S1 = (np.log(Sigmoid))*Y[i]
        S2 = 1 - Sigmoid
        S2 = (np.log(S2))*(1-Y[i])
        S = S1+S2 
        J += S
          
    J = J/(-1*m) 

    return J


In [496]:
def gradientDescent(X, Y,thetas,alpha, n_epoch):
    
    m = len(Y)
    n = len(X[0])
    J = list()
    Sum0 = 0 
    Sum = np.zeros([len(X[0]),1], dtype = float) 
    
    for epoch in range(n_epoch):
        j = 0
        k = 0
        Sum0 = 0
    
        for j in range(n):
            i = 0
            Sum[j] = 0
            
            for i in range(m):
                dot = np.dot(X[i],thetas)
                #print('dot',dot)
                Sum[j] = Sum[j]+((sigmoid(dot))-Y[i])*X[i][j]
            Sum[j] = Sum[j]/m
            Sum[j] =  thetas[j] - Sum[j]*alpha
            
            
                           
        for k in range(n):
            thetas[k] = Sum[k]
            
            
        #print(lossfunt(X, Y, thetas))
        J.append(lossfunt(X, Y, thetas))
    return thetas, J    
  

In [514]:
n_epoch = 100
alpha = 0.01

thetas = np.zeros([len(X[0]),1], dtype = float)   
thetas,J = gradientDescent(X ,Y,thetas,alpha, n_epoch)
print('Thetas are',thetas)
print('Cost is ',J[-1])

Thetas are [[-1.11492092e-09]
 [ 1.17034842e-02]
 [-8.90301086e-03]
 [ 4.64485062e-02]
 [ 8.22929049e-04]
 [-6.53218659e-03]
 [ 7.71440514e-05]]
Cost is  [0.66900994]


In [498]:
def preprocessingTestData(datapath,Y):
    
    numberofpositives = []
    numberofnegatives =[]
    rating = []
    processeddata = []
    isNO = []
    isEXC =[]
    count = []
    output = []
    for file in os.listdir(datapath):
           # print(file)
            fileopened = open(datapath+ "/" + file, "r",encoding='ISO-8859-1')
            text = fileopened.read()
            strings = text.split(' ')

            numberofpositives.append((len(findStrings(strings,pos_vocab))))
            numberofnegatives.append(len(findStrings(strings,neg_vocab)))
            ratingsplit = (file.split("_")[1])
            rating.append(ratingsplit.split(".")[0])
            isNO.append(findNO(strings))
            isEXC.append(findEXC(strings))
            count.append(len(strings))
            output.append(Y)
            

            dataset = np.vstack([numberofpositives,numberofnegatives,rating,count,isNO,isEXC,output]).T
           
    
    fileopened.close()
    
    return dataset
  

In [499]:
#print('Positives Test data')
testdatasetpos = preprocessingTestData("Dataset/test/pos","1")
#print('negatives Test data')
testdatasetneg = preprocessingTestData("Dataset/test/neg","0")
testdatasetall = np.concatenate((testdatasetpos, testdatasetneg))
print(testdatasetall)

[['6' '7' '9' ... '1' '0' '1']
 ['3' '6' '9' ... '0' '0' '1']
 ['3' '0' '9' ... '0' '0' '1']
 ...
 ['9' '9' '1' ... '1' '0' '0']
 ['8' '8' '3' ... '1' '0' '0']
 ['1' '0' '1' ... '0' '0' '0']]


In [500]:
#predict 
def predict(X = [],thetas = []):
    h_x = 0.0
    l = len(X)
    predicted_Y = []
    for i in range (0,l):

        h_x = np.dot(X[i],thetas) 
        h_x = sigmoid(h_x)
        
        if h_x > 0.5:
            predicted_Y.append(1)
        else:
            predicted_Y.append(0)
            
            
    return  predicted_Y

In [515]:
test_X = testdatasetall[:, 0:6]
test_X = test_X.astype(np.float)

Actual_Y = testdatasetall[:, 6]


#normalization
for i in range (0,len(test_X)):
    
    for j in range(0,6):
        test_X[i][j]= (test_X[i][j]-means_X[j])/sd_X[j] 
        
        

X0 = np.ones((len(test_X),1))    
test_X = np.hstack((X0,test_X))

predicted_Y = predict(test_X,thetas)

In [516]:
#confusion Matrix

def confusionMatrix(predicted_Y, Actual_Y):
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for i in range(len(Actual_Y)): 
        if predicted_Y[i] ==1 and Actual_Y[i]==1 :
           tp += 1
        if predicted_Y[i]==1 and Actual_Y[i]== 0:
           fp += 1
        if predicted_Y[i]==0 and Actual_Y[i] == 0:
           tn += 1
        if predicted_Y[i]==0 and Actual_Y[i] ==1 :
           fn += 1

    return(tp, tn, fp, fn)


In [517]:
Actual_Y = Actual_Y.astype(np.float)
tp, tn, fp, fn = confusionMatrix(predicted_Y,Actual_Y)
print(tp, tn, fp, fn)
matrix = np.array([[tp,fp], [fn,tn]])
print(matrix)

12459 12458 42 41
[[12459    42]
 [   41 12458]]


In [518]:
#parameters
precision = tp/(tp+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
recall = tp/(tp+fn)
F1_score = ((precision*recall)/(precision+recall))*2
print('precision is :',precision)
print('accuracy is :',accuracy)
print('recall is :',recall)
print('F1_score is :',F1_score)


precision is : 0.9966402687784978
accuracy is : 0.99668
recall is : 0.99672
F1_score is : 0.9966801327946884


In [538]:
#stochastic
np.random.shuffle(traindatasetall)
X, Y = traindatasetall[:, 0:6], traindatasetall[:, 6]
#print(X
#print(Y)
X = X.astype(np.float)
Y = Y.astype(np.float)
#print(Y)

#normalization
means_X = np.mean(X,axis=0)
sd_X = np.std(X,axis = 0)
for i in range (0,len(X)):
    
    for j in range(0,6):
        X[i][j]= (X[i][j]-means_X[j])/sd_X[j] 

X0 = np.ones((len(X),1))
X = np.hstack((X0,X))

#print(X)
#print(Y)


In [534]:
#stochastic 
def stochasticGradientDescent(X, Y,thetas, alpha, n_epoch):
    
    m = len(Y)
    m = m
    n = len(X[0])
    J1 = list() 
    Sum0 = 0 
    Sum = np.zeros([len(X[0]),1], dtype = float) 
    
    for epoch in range(n_epoch):
            
        for i in range(m):
            dot = np.dot(X[i],thetas)
            sig = sigmoid(dot)
            #print('iteration is',i)
               
            for j in range(n):
                Sum[j] = (sig-Y[i])*X[i][j]
                thetas[j] =  thetas[j] - Sum[j]*alpha
            J1.append(lossfunt(X, Y,thetas))
            #print(lossfunt(X, Y,thetas))
   
    return thetas, J1

n_epoch = 100
alpha = 0.01
thetas = np.zeros([len(X[0]),1], dtype = float)
   
thetas1,J1 = stochasticGradientDescent(X ,Y, thetas,alpha, n_epoch)
print('Stochastic Thetas are',thetas1)
print('Cost is ',J1[-1])
  

Stochastic Thetas are [[ 0.13294184]
 [ 1.01640674]
 [-0.82773029]
 [ 4.796607  ]
 [ 0.12932396]
 [-0.26526963]
 [-0.15150485]]
Cost is  [0.02346442]


In [535]:
test_X = testdatasetall[:, 0:6]
test_X = test_X.astype(np.float)

Actual_Y = testdatasetall[:, 6]

#normalization
for i in range (0,len(test_X)):
    
    for j in range(0,6):
        test_X[i][j]= (test_X[i][j]-means_X[j])/sd_X[j] 
        
        

X0 = np.ones((len(test_X),1))    
test_X = np.hstack((X0,test_X))

predicted_Y = predict(test_X,thetas1)

In [539]:
Actual_Y = Actual_Y.astype(np.float)
tp, tn, fp, fn = confusionMatrix(predicted_Y,Actual_Y)
print(tp, tn, fp, fn)
matrix = np.array([[tp,fp], [fn,tn]])
print(matrix)

12491 12458 42 9
[[12491    42]
 [    9 12458]]


In [540]:
#parameters
precision = tp/(tp+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
recall = tp/(tp+fn)
F1_score = ((precision*recall)/(precision+recall))*2
print('precision is :',precision)
print('accuracy is :',accuracy)
print('recall is :',recall)
print('F1_score is :',F1_score)

precision is : 0.9966488470438044
accuracy is : 0.99796
recall is : 0.99928
F1_score is : 0.9979626892501897
